# Imports

In [7]:
import os
import gc
import requests
from datetime import datetime
import pandas as pd
from datetime import timedelta
import requests
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import time
import ast
import json
import numpy as np 
fmp = 'your_api_key' 

# function that makes requests to the api and ensures that code won't stop in the middle if the problem is on the api side
def req(r):
    deaths = -5
    while True:
        if not deaths:
            deaths = -5
            time.sleep(1)
        got = requests.get(r).content
        try:
            got = json.loads(got.decode('utf-8'))
            deaths = -5
            time.sleep(.2)
        except:
            deaths += 1
            continue
        if 'Error Message' in got:
            deaths += 1
            continue
        return got
import pandas as pd

# Collecting Earnings Call transcripts

First of all we need to collect Earnings Call transcripts of NYSE and NASDAQ traded stocks

In [ ]:
#request all availible symbols from the api and keeping only NYSE and NASDAQ traded stocks
df = pd.DataFrame(req(f'https://financialmodelingprep.com/api/v3/stock/list?apikey={fmp}'))
df = df[(df.exchange == 'New York Stock Exchange') | (df.exchange == 'NASDAQ')]
df = df[df['type'] == 'stock']
df['ticker'] = df.symbol
df = df[df.ticker.apply(lambda x: '-'  not in x)].drop_duplicates(subset = 'name', keep = 'first')

In [ ]:
#collecting transcripts for several years
yer, quat, trans, dtm = [], [], [], []
new = {'ticker':[], 'year': [], 'quarter': [], 'transcript': [], 'time':[]}
for i in tqdm(range(len(df))):
    for j in range(2016, 2023):
        for k in range(1, 5):
            cll = req(f'https://financialmodelingprep.com/api/v3/earning_call_transcript/{df.ticker.iloc[i]}?quarter={k}&year={j}&apikey={fmp}')
            if len(cll) == 0: continue
            else:
                new['ticker'].append(df.ticker.iloc[i])
                new['year'].append(j)
                new['quarter'].append(k)
                new['transcript'].append(cll[0]['content'])
                new['time'].append(cll[0]['date'])
df = pd.DataFrame(new).dropna()
df['time'] = pd.to_datetime(df.time)

100%|██████████| 2848/2848 [9:03:59<00:00, 11.46s/it]  


# Data cleaning

We need to clean the text by removing phrases that don't help the model understand the company's prospects. For instance, we should remove all phrases spoken by the call operator and any phrases that include greetings and goodbyes made during the call.

In [ ]:
ph = ["hi", "hello", "hey", "good morning", "good day", "good afternoon", "good evening", "greetings", "greeting", "good to see you", "its good seeing you", "how are you", "how're you", "how are you doing", "how ya doin'", "how ya doin", "how is everything", "how is everything going", "how's everything going", "how is you", "how's you", "how are things", "how're things", "how is it going", "how's it going", "how's it goin'", "how's it goin", "how is life been treating you", "how's life been treating you", "how have you been", "how've you been", "what is up", "what's up", "what is cracking", "what's cracking", "what is good", "what's good", "what is happening", "what's happening", "what is new", "what's new", "what is neww", "g’day", "howdy", "welcome"]
gc.collect()
import re
def del_txt(x):
    new = []
    for i in x.lower().split('\n')[1:]:
        new_str = ''
        if len(i) == 0: continue
        for j in range(min(100, len(i))):
            if i[j] == ':': break
        if i[:j] == 'operator': continue
        if j != len(i)+1: i = i[j+1:]
        start = 0
        for j in re.finditer('\.|\?|\;', i):
            if i[j.start()] == '?': 
                start = j.start()+1
                continue
            st = i[start:j.start()+1]
            good = True
            for p in ph:
                if re.search(f'{p}[ .,\d]', st) != None:
                    good = False; break
            if st.count(' ') <= 10:
                if re.search(f'thank you[ .,\d]', st) != None: good = False
            if good: new_str += st
            start = j.start()+1
        new.append(new_str)
    return('\n'.join(new))
for i in range(len(df)):
    df.transcript.iloc[i] = del_txt(df.transcript.iloc[i])
gc.collect()

0

# Stock price collection

Now we need to collect market capitalization and prices. We will use the capitalization last known before the Earnings Call date, as it must be known in advance. For price change which will be the target of our model, we will use the ratio of the closing price on the call date plus 90 days to the closing price on the call date plus 1 day. This allows time to make purchases after the prediction.

In [ ]:
mass = []
for i in tqdm(range(len(df))):
    prc = req(f"https://financialmodelingprep.com/api/v3/historical-price-full/{df.ticker.iloc[i]}?from={df.time.iloc[i] - timedelta(days = 5)}&to={df.time.iloc[i] - timedelta(days = 1)}&limit=1000000000&apikey={fmp}")
    shrs = req(f"https://financialmodelingprep.com/api/v3/quote/{df.ticker.iloc[i]}?apikey={fmp}")
    if (len(prc) == 0) or (len(shrs) == 0): 
        mass.append(np.nan)
        continue
    shrs = shrs[0]['sharesOutstanding']
    prc = prc['historical'][0]['close']
    mass.append(prc*shrs/1000000000)
df['mcap'] = mass

 99%|█████████▉| 44901/45208 [11:13:44<04:05,  1.25it/s]  

In [ ]:
df.to_pickle('transcripts.pkl')

In [ ]:
diff = []
df['time'] = pd.to_datetime(df.time)
for i in tqdm(range(0, len(df))):
    prc = req(f'https://financialmodelingprep.com/api/v3/historical-price-full/{df.ticker.iloc[i]}?from={df.time.iloc[i] + timedelta(days = 1)}&to={df.time.iloc[i] + timedelta(days = 90)}&apikey={fmp}')
    if len(prc) == 0:
        diff.append(np.nan)
        continue
    con = prc['historical'][-1]['close']
    prc = prc['historical'][0]['close']
    if prc == 0:
        diff.append(np.nan)
        continue
    diff.append(con/prc)
df['change'] = diff

100%|██████████| 45208/45208 [6:52:32<00:00,  1.83it/s]   


In [ ]:
df.to_pickle('transcripts.pkl')

# Collecting financials data

Now we need to collect data about the company's financials to enhance our NLP model with another model using the results of the NLP model and financial data. We need to gather all financials reported before the earnings call. Additionally, we need to standardize the data across companies. Since the data is reported in dollars and companies vary in size, we can't use it as is. Instead, we will divide all financials for the current quarter by the same metrics from the previous year to obtain year-to-year changes in each column. Using the same quarter from a year ago helps us avoid seasonality effects that could arise if we used just the previous quarter. 

However, challenges arise when at least one of the two quarters, either the current or the one from a year ago, has a negative number. In such cases, traditional percentage change calculations become unfeasible. To address this, our model employs specialized formulas designed for each of these scenarios. These formulas allow the AI to get a single numeric representation that indicates the change in a given metric, even when faced with negative values in the data. You can find formulas in the code.



In [ ]:
df = pd.read_pickle('transcripts.pkl')

In [ ]:
# creating dictionary containing names of all possible financials
cash1 = req(f"https://financialmodelingprep.com/api/v3/cash-flow-statement/AAPL?period=quarter&limit=1000000000&apikey={fmp}")[0]
balance1 = req(f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/AAPL?period=quarter&limit=1000000000&apikey={fmp}")[0]
incm1 = req(f"https://financialmodelingprep.com/api/v3/income-statement/AAPL?period=quarter&limit=1000000000&apikey={fmp}")[0]
dic = set(['ticker','quarter','year','time'])
for i in (cash1, balance1, incm1): dic.update(i.keys())
for i in ('date', 'symbol', 'reportedCurrency', 'fillingDate', 'acceptedDate', 'period', 'link', 'finalLink'): dic.remove(i)
dic = {i:[] for i in dic}

In [ ]:
from tqdm.contrib.telegram import tqdm as tqdm_tg

cache_cash, cache_peers, cache_shrs, cache_balance, cache_incm = {}, {}, {}, {}, {}
PFCF_discount, EVE_discount, PS_discount, PE_discount = [], [], [], []
df.quarter = df.quarter.astype(str)
df.year = df.year.astype(str)
for i in tqdm(range(len(df))): 
    if df.ticker.iloc[i] in cache_cash: cash = cache_cash[df.ticker.iloc[i]]
    else: 
        #creating cache so we don;t have to make the same request to api several times
        cash = req(f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{df.ticker.iloc[i]}?period=quarter&limit=1000000000&apikey={fmp}")
        cache_cash[df.ticker.iloc[i]] = cash
    #finding needed quarter and year in cash flow statement
    for j in cash:
        if (j['period'] == 'Q' + df.quarter.iloc[i]) & (j['calendarYear'] == df.year.iloc[i]):
            break
    if not ((j['period'] == 'Q' + df.quarter.iloc[i]) & (j['calendarYear'] == df.year.iloc[i])):
        continue
    cash1 = j
    #finding needed quarter in a previous year
    for j in cash:
        if (j['period'] == 'Q' +str(df.quarter.iloc[i])) and (j['calendarYear'] == str(int(cash1['calendarYear'])-1)):
            break
    if not ((j['period'] == 'Q' +str(df.quarter.iloc[i])) and (j['calendarYear'] == str(int(cash1['calendarYear'])-1))):
        continue
    cash2 = j
    if type(pd.to_datetime(cash1['acceptedDate'])) == type(None): 
        continue    
    #doing the same for balance sheet statement
    if df.ticker.iloc[i] in cache_balance: balance = cache_balance[df.ticker.iloc[i]]
    else:
        balance = req(f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{df.ticker.iloc[i]}?period=quarter&limit=1000000000&apikey={fmp}")
        cache_balance[df.ticker.iloc[i]] = balance

    for j in balance:
        if (j['period'] == 'Q' + df.quarter.iloc[i]) & (j['calendarYear'] == df.year.iloc[i]):
            break
    if not ((j['period'] == 'Q' + df.quarter.iloc[i]) & (j['calendarYear'] == df.year.iloc[i])):
        continue
    balance1 = j
    for j in balance:
        if (j['period'] == 'Q' +str(df.quarter.iloc[i])) and (j['calendarYear'] == str(int(balance1['calendarYear'])-1)):
            break
    if not ((j['period'] == 'Q' +str(df.quarter.iloc[i])) and (j['calendarYear'] == str(int(balance1['calendarYear'])-1))):
            continue
    balance2 = j

    #doing the same for income statement

    if df.ticker.iloc[i] in cache_incm: incm = cache_incm[df.ticker.iloc[i]]
    else:
        incm = req(f"https://financialmodelingprep.com/api/v3/income-statement/{df.ticker.iloc[i]}?period=quarter&limit=1000000000&apikey={fmp}")

    for j in incm:
        if (j['period'] == 'Q' + df.quarter.iloc[i]) & (j['calendarYear'] == df.year.iloc[i]):
            break
    if not ((j['period'] == 'Q' + df.quarter.iloc[i]) & (j['calendarYear'] == df.year.iloc[i])):
        continue
    incm1 = j

    for j in incm:
        if (j['period'] == 'Q' +str(df.quarter.iloc[i])) and (j['calendarYear'] == str(int(incm1['calendarYear'])-1)):
            break
    if not ((j['period'] == 'Q' +str(df.quarter.iloc[i])) and (j['calendarYear'] == str(int(incm1['calendarYear'])-1))):
        continue
    incm2 = j
    
    # some parameters can appear in several statements (for example in income statement and balance sheet statement)
    # so we need to check whether we already added it.
    for k in dic:
        if (k in cash1.keys()):
            try:
                if cash2[k] != 0:
                    # just taking ratio if both of the numbers are positive
                    if (float(cash1[k]) > 0) and (float(cash2[k]) > 0):
                        dic[k].append(float(cash1[k])/float(cash2[k]))
                        
                    # In the case where both numbers are negative, we have two different cases: 
                    # when the absolute value of the current quarter is less and more than that of the previous one. 
                    elif (float(cash1[k]) < 0) and (float(cash2[k]) < 0):
        
                    # In the first case, we divide the current quarter value by the previous one and subtract it from 1. 
                    # (than adding one to maintain the same format where 1 means that number hasn't changed)
                    # For example, if a company had 100 million in loss a year ago and only 50 million now we get 
                    # 100%*(1-(-50)/(-100)) = 50% increase in net income.
                        if float(cash1[k]) > float(cash2[k]):
                            dic[k].append(1+(1 - (abs(float(cash1[k]))/abs(float(cash2[k])))))
                            
                    # In the case where the absolute value of the current quarter is higher, 
                    # we take the negative of the current quarter over the previous one. 
                    # Thus we can find how much the loss in this metric increased also showing the model negative nature of this decrease. 
                    # For example, if a company had 50 million in loss a year ago and 100 million now we get 
                    # 100%*(-(-100)/(-50)-1) = -300% in net income.
                        elif float(cash1[k]) < float(cash2[k]):
                            dic[k].append(-(float(cash1[k])/float(cash2[k])))
                            
                    # when they're equal we just take 1 
                        else:
                            dic[k].append(1)
                    
                    # Here we begin by taking the absolute value of the previous quarter’s metric as a baseline. 
                    # By doing so, we essentially increase the previous quarter’s value by twice its absolute value. 
                    # Therefore, to calculate fair change we should add two absolute values of the previous quarter to the current one. 
                    # Thus, we use the following formula: 100%*((current_quarter_value + 2*|previous_quarter_value|)/|previous_quarter_value| – 1)
                    # For example, if a company has 100 million net income this quarter, but had 50 million in loss a year ago, 
                    # we get 100%*((100 + 2*|-50|)/|-50| – 1) = 300% increase in net income
                    
                    elif (float(cash1[k]) > 0) and (float(cash2[k]) < 0):
                        dic[k].append((float(cash1[k])+2*abs(float(cash2[k])))/abs(float(cash2[k])))
            
                    # In the last case where current quarter is negative and previous one is positive
                    # we divide the current quarter’s value by the previous one and then subtract an additional 1 
                    # from the result. This process yields a negative percentage value that surpasses -100%.
                    # For example, if a company has 50 million in a loss in this quarter, but had 100 million in net income a year ago, 
                    # we get 100%*(-50/100-2) = -250% decrease in net income
                    else:
                        dic[k].append((float(cash1[k])/float(cash2[k]))-1)
                else:
                    if cash1[k] == 0:
                        dic[k].append(1)
                    elif cash1[k] > 0:
                        dic[k].append(2)
                    else:
                        dic[k].append(-2)
            except: 
                dic[k].append(np.nan)
    # doing the same for balance sheet statement
    for k in dic:
        if (k in balance1.keys()) and (k not in cash1.keys()):
            try: 
                if balance2[k] != 0:
                    if (float(balance1[k]) > 0) and (float(balance2[k]) > 0):
                        dic[k].append(float(balance1[k])/float(balance2[k]))
                    elif (float(balance1[k]) < 0) and (float(balance2[k]) < 0):
                        if float(balance1[k]) > float(balance2[k]):
                            dic[k].append(1+(1 - (abs(float(balance1[k]))/abs(float(balance2[k])))))
                        elif float(balance1[k]) < float(balance2[k]):
                            dic[k].append(-(float(balance1[k])/float(balance2[k])))
                        else:
                            dic[k].append(1)
                    elif (float(balance1[k]) > 0) and (float(balance2[k]) < 0):
                        dic[k].append((float(balance1[k])+2*abs(float(balance2[k])))/abs(float(balance2[k])))
                    else:
                        dic[k].append((float(balance1[k])/float(balance2[k]))-1)
                else:
                    if balance1[k] == 0:
                        dic[k].append(1)
                    elif balance1[k] > 0:
                        dic[k].append(2)
                    else:
                        dic[k].append(-2)
 
            except: 
                dic[k].append(np.nan)
    # doing the same for income statement
    for k in dic:
        if (k in incm1.keys() and (k not in cash1.keys())) and (k not in balance1.keys()):
            try: 
                if incm2[k] != 0:
                    if (float(incm1[k]) > 0) and (float(incm2[k]) > 0):
                        dic[k].append(float(incm1[k])/float(incm2[k]))
                    elif (float(incm1[k]) < 0) and (float(incm2[k]) < 0):
                        if float(incm1[k]) > float(incm2[k]):
                            dic[k].append(1+(1 - (abs(float(incm1[k]))/abs(float(incm2[k])))))
                        elif float(incm1[k]) < float(incm2[k]):
                            dic[k].append(-(float(incm1[k])/float(incm2[k])))
                        else:
                            dic[k].append(1)
                    elif (float(incm1[k]) > 0) and (float(incm2[k]) < 0):
                        dic[k].append((float(incm1[k])+2*abs(float(incm2[k])))/abs(float(incm2[k])))
                    else:
                        dic[k].append((float(incm1[k])/float(incm2[k]))-1)
                else:
                    if incm1[k] == 0:
                        dic[k].append(1)
                    elif incm1[k] > 0:
                        dic[k].append(2)
                    else:
                        dic[k].append(-2)
            except:
                dic[k].append(np.nan)
    dic['ticker'].append(df.ticker.iloc[i])
    dic['quarter'].append(df.quarter.iloc[i])
    dic['year'].append(df.year.iloc[i])
    dic['time'].append(pd.to_datetime(cash1['acceptedDate']))

 78%|███████▊  | 35416/45208 [4:50:39<59:38,  2.74it/s]   

In [ ]:
df = pd.DataFrame(dic)
df['time'] = pd.to_datetime(df.time).apply(lambda x: x.date())

In [ ]:
df.to_pickle('financials_data.pkl')